In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def load_food_data(filepath):
    """ CSV 파일에서 음식 데이터를 불러오는 함수 """
    df = pd.read_csv(filepath)
    df["칼로리"] = df["칼로리"].astype(float)
    df["탄수화물"] = df["탄수화물"].astype(float)
    df["단백질"] = df["단백질"].astype(float)
    df["지방"] = df["지방"].astype(float)
    return df

def get_nutritional_targets(sex, body_type, goal):
    """ 사용자 입력에 따른 목표 칼로리 및 탄단지 비율 설정 """
    calorie_dict = {
        "사과형": {"diet": {"남성": 1500, "여성": 1300}, "muscle_gain": {"남성": 2500, "여성": 2200}},
        "배형": {"diet": {"남성": 1600, "여성": 1400}, "muscle_gain": {"남성": 2600, "여성": 2300}},
    }
    macro_ratios = {  # 탄:단:지 비율
        "diet": (50, 30, 20),
        "muscle_gain": (40, 40, 20),
        "maintain": (45, 35, 20)
    }
    total_calories = calorie_dict.get(body_type, {}).get(goal, {}).get(sex, 1800)
    carb_ratio, protein_ratio, fat_ratio = macro_ratios[goal]
    return total_calories, carb_ratio, protein_ratio, fat_ratio

def optimize_meal_plan(food_df, total_calories, macro_ratios):
    """ 최적화 문제를 풀어 탄단지 맞춘 식단 추천 """
    carb_target, protein_target, fat_target = [r / 100 * total_calories / 4 for r in macro_ratios]
    calorie_target = total_calories
    
    food_nutrients = food_df[["칼로리", "탄수화물", "단백질", "지방"]]
    num_foods = len(food_df)
    
    # 목적함수 계수 설정 (총 오차 최소화)
    c = np.ones(num_foods)
    
    # 제약 조건
    A_eq = np.array([
        food_nutrients["칼로리"].values,
        food_nutrients["탄수화물"].values,
        food_nutrients["단백질"].values,
        food_nutrients["지방"].values
    ])
    b_eq = np.array([calorie_target, carb_target, protein_target, fat_target])
    
    # 각 음식의 섭취량이 0 이상
    bounds = [(0, None)] * num_foods
    
    # 최적화 실행
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    
    if res.success:
        selected_foods = food_df.iloc[np.where(res.x > 0.01)]
        return selected_foods
    else:
        return None
        

# 실행 예시
file_path = "data/food_nutrition_data.csv"
food_data = load_food_data(file_path)
total_cal, c_ratio, p_ratio, f_ratio = get_nutritional_targets("남성", "사과형", "diet")
food_data = food_data.fillna(0)  # NaN을 0으로 채우기
recommended_meal = optimize_meal_plan(food_data, total_cal, (c_ratio, p_ratio, f_ratio))
print(recommended_meal)


    식품명(한글)     칼로리     탄수화물     단백질      지방 1회 섭취량
103   당근케이크  6952.0   902.40   65.77  355.78  1704g
118    된장찌개  6857.0  1556.43  186.11    9.14  3265g
169     명란젓  1445.0    16.77  190.00   82.41   745g
462     코코아   229.0    54.30   19.60   13.70   100g


In [21]:
def format_meal_plan(meal_df):
    """
    추천된 식단을 아침, 점심, 저녁으로 구분하여 보기 좋게 출력하는 함수
    """
    meal_df = meal_df.sort_values(by="칼로리", ascending=False).reset_index(drop=True)
    total_calories = meal_df["칼로리"].sum()
    
    # 아침, 점심, 저녁 비율 설정 (아침 30%, 점심 40%, 저녁 30%)
    meal_ratios = {"아침": 0.3, "점심": 0.4, "저녁": 0.3}
    meal_plan = {"아침": [], "점심": [], "저녁": []}
    
    # 각 끼니별 목표 칼로리
    meal_targets = {meal: total_calories * ratio for meal, ratio in meal_ratios.items()}
    
    for meal_type in ["아침", "점심", "저녁"]:
        remaining_calories = meal_targets[meal_type]
        
        for idx, row in meal_df.iterrows():
            if row["칼로리"] <= remaining_calories:
                meal_plan[meal_type].append(row)
                remaining_calories -= row["칼로리"]
    
    # 출력 형식 정리
    print("\n===== 추천 식단 =====")
    for meal, foods in meal_plan.items():
        print(f"\n{meal}:")
        for food in foods:
            print(f"- {food['식품명(한글)']} (1회 섭취량: {food['1회 섭취량']}, 칼로리: {food['칼로리']}kcal, 탄: {food['탄수화물']}g, 단: {food['단백질']}g, 지: {food['지방']}g)")
    
    return meal_plan


In [24]:
format_meal_plan(recommended_meal)


===== 추천 식단 =====

아침:
- 명란젓 (1회 섭취량: 745g, 칼로리: 1445.0kcal, 탄: 16.77g, 단: 190.0g, 지: 82.41g)
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)

점심:
- 명란젓 (1회 섭취량: 745g, 칼로리: 1445.0kcal, 탄: 16.77g, 단: 190.0g, 지: 82.41g)
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)

저녁:
- 명란젓 (1회 섭취량: 745g, 칼로리: 1445.0kcal, 탄: 16.77g, 단: 190.0g, 지: 82.41g)
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)


{'아침': [식품명(한글)       명란젓
  칼로리        1445.0
  탄수화물        16.77
  단백질         190.0
  지방          82.41
  1회 섭취량       745g
  Name: 2, dtype: object,
  식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 3, dtype: object],
 '점심': [식품명(한글)       명란젓
  칼로리        1445.0
  탄수화물        16.77
  단백질         190.0
  지방          82.41
  1회 섭취량       745g
  Name: 2, dtype: object,
  식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 3, dtype: object],
 '저녁': [식품명(한글)       명란젓
  칼로리        1445.0
  탄수화물        16.77
  단백질         190.0
  지방          82.41
  1회 섭취량       745g
  Name: 2, dtype: object,
  식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 3, dtype: object]}

In [33]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def get_user_input():
    """ 사용자로부터 성별, 체형, 목표 입력 받기 """
    gender = input("성별을 입력하세요 (남성/여성): ")
    body_type = input("체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형): ")
    goal = input("목표를 입력하세요 (diet/maintain/gain): ")
    return gender, body_type, goal

def optimize_meal_plan(food_df, total_calories, macro_ratios):
    """
    총 칼로리 및 탄단지 비율을 고려하여 최적의 식단을 구성하는 함수
    """
    c_ratio, p_ratio, f_ratio = macro_ratios
    num_foods = len(food_df)
    
    # 목적 함수 (칼로리 최소화)
    c = food_df["칼로리"].values
    
    # 제약 조건 설정 (탄수화물, 단백질, 지방 비율)
    A_eq = [
        food_df["탄수화물"].values,
        food_df["단백질"].values,
        food_df["지방"].values,
        food_df["칼로리"].values
    ]
    b_eq = [
        total_calories * (c_ratio / 100) / 4,
        total_calories * (p_ratio / 100) / 4,
        total_calories * (f_ratio / 100) / 9,
        total_calories
    ]
    
    # 각 음식 최소 선택량 (음식이 포함될 수 있도록 0이 아닌 값 설정)
    bounds = [(0, 1)] * num_foods
    
    # 최적화 실행
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    
    if res.success:
        selected_foods = food_df.iloc[np.where(res.x > 0.01)]
        return selected_foods
    else:
        print("최적화 실패. 기본 식단을 반환합니다.")
        return food_df.sample(n=3)  # 기본 샘플 식단 제공

def format_meal_plan(meal_df, total_calories, c_ratio, p_ratio, f_ratio):
    """
    추천된 식단을 아침, 점심, 저녁으로 구분하여 보기 좋게 출력하는 함수 (탄단지 고려)
    """
    meal_ratios = {"아침": 0.3, "점심": 0.4, "저녁": 0.3}
    meal_plan = {"아침": [], "점심": [], "저녁": []}
    meal_targets = {meal: total_calories * ratio for meal, ratio in meal_ratios.items()}
    
    for meal_type in ["아침", "점심", "저녁"]:
        remaining_calories = meal_targets[meal_type]
        remaining_carb = c_ratio / 100 * total_calories / 4 * meal_ratios[meal_type]
        remaining_protein = p_ratio / 100 * total_calories / 4 * meal_ratios[meal_type]
        remaining_fat = f_ratio / 100 * total_calories / 9 * meal_ratios[meal_type]
        
        shuffled_df = meal_df.sample(frac=1).reset_index(drop=True)  # 음식 랜덤 배정
        for idx, row in shuffled_df.iterrows():
            if (row["칼로리"] <= remaining_calories and 
                row["탄수화물"] <= remaining_carb and
                row["단백질"] <= remaining_protein and
                row["지방"] <= remaining_fat):
                meal_plan[meal_type].append(row)
                remaining_calories -= row["칼로리"]
                remaining_carb -= row["탄수화물"]
                remaining_protein -= row["단백질"]
                remaining_fat -= row["지방"]
            if len(meal_plan[meal_type]) == 0:
                meal_plan[meal_type].append(shuffled_df.iloc[0])  # 최소 한 개 음식 포함
    
    print("\n===== 추천 식단 =====")
    for meal, foods in meal_plan.items():
        print(f"\n{meal}:")
        for _, food in pd.DataFrame(foods).iterrows():
            print(f"- {food['식품명(한글)']} (1회 섭취량: {food['1회 섭취량']}, 칼로리: {food['칼로리']}kcal, 탄: {food['탄수화물']}g, 단: {food['단백질']}g, 지: {food['지방']}g)")
    
    return meal_plan

# 사용자 입력 받기
gender, body_type, goal = get_user_input()

# 예시: 사용자 입력을 이용해 최적화 함수 실행 (최적화 함수는 별도로 구현 필요)
# total_cal, c_ratio, p_ratio, f_ratio = get_nutritional_targets(gender, body_type, goal)
# recommended_meal = optimize_meal_plan(food_data, total_cal, (c_ratio, p_ratio, f_ratio))
# format_meal_plan(recommended_meal, total_cal, c_ratio, p_ratio, f_ratio)


성별을 입력하세요 (남성/여성):  남성
체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형):  사과형
목표를 입력하세요 (diet/maintain/gain):  diet



===== 추천 식단 =====

아침:
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)

점심:
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)

저녁:
- 된장찌개 (1회 섭취량: 3265g, 칼로리: 6857.0kcal, 탄: 1556.43g, 단: 186.11g, 지: 9.14g)
- 코코아 (1회 섭취량: 100g, 칼로리: 229.0kcal, 탄: 54.3g, 단: 19.6g, 지: 13.7g)


{'아침': [식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 0, dtype: object],
 '점심': [식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 0, dtype: object],
 '저녁': [식품명(한글)       된장찌개
  칼로리         6857.0
  탄수화물       1556.43
  단백질         186.11
  지방            9.14
  1회 섭취량       3265g
  Name: 0, dtype: object,
  식품명(한글)      코코아
  칼로리        229.0
  탄수화물        54.3
  단백질         19.6
  지방          13.7
  1회 섭취량      100g
  Name: 3, dtype: object]}